<a href="https://colab.research.google.com/github/Idk507/Qwen_2_VL/blob/main/MultiModel_RAG_with_Qwen2_and_ColPali.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install git+https://github.com/huggingface/transformers accelerate flash_attn
!pip install qwen_vl_utils av

!pip install --upgrade git+https://github.com/huggingface/transformers.git byaldi accelerate flash-attn qwen_vl_utils pdf2image
!sudo apt-get install -y poppler-utils

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-v_sjqs4_
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-v_sjqs4_
  Resolved https://github.com/huggingface/transformers to commit e782e95e3465b66a377c0a7fe95f8f10cd87459d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-9p7iyj3r
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-9p7iyj3r
  Resolved https://github.com/huggingface/transformers.git to commit e782e95e3465b66a377c0a7fe95f8f10cd87459d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached ninja-1.11.1.1-py2.py3-none-manylinux1_x86_64.manylinux_2_5_x86_64.

In [19]:
!pip install --upgrade transformers bitsandbytes accelerate flash-attn peft sentencepiece datasets trl sentence-transformers

In [21]:
from byaldi import RAGMultiModalModel
from transformers import Qwen2VLForConditionalGeneration,AutoTokenizer, AutoProcessor,AutoModelForCausalLM,AutoTokenizer,set_seed
from qwen_vl_utils import process_vision_info
import torch
from pdf2image import convert_from_path
import os
import multiprocessing
from trl import SFTTrainer,SFTConfig

from datasets import load_dataset

In [22]:
model_name = "Qwen/Qwen2.5-7B"

tokenizer = AutoTokenizer.from_pretrained(model_name,trust_remote_code=True)
tokenizer.pad_token = "<|im_end|>"
tokenizer.pad_token_id = 151645

tokenizer.padding_side = "left"

tokenizer_config.json:   0%|          | 0.00/7.23k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [24]:
ds = load_dataset("timdettmers/openassistant-guanaco")




Repo card metadata block was not found. Setting CardData to empty.


In [25]:
ds = ds['train'].select(range(1000))

In [26]:
ds

Dataset({
    features: ['text'],
    num_rows: 1000
})

In [27]:
def process(row):
  row["text"] = row["text"]+tokenizer.eos_token
  return row

ds = ds.map(process,num_proc=multiprocessing.cpu_count(),load_from_cache_file=False)

Map (num_proc=2):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [29]:
!pip install -U bitsandbytes

In [30]:
model = AutoModelForCausalLM.from_pretrained(model_name,trust_remote_code=True,device_map="auto",load_in_8bit=True)

model.gradient_checkpointing_enable(gradient_checkpointing_kwargs={"use_reentrant":False})

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


ImportError: Using `bitsandbytes` 8-bit quantization requires the latest version of bitsandbytes: `pip install -U bitsandbytes`

In [ ]:
training_arguments = SFTConfig(
    output_dir="./results",
    eval_strategy="steps",
    do_eval = True,
    optim = "paged_adaw_8bit",
    per_device_train_batch_size = 1,
    per_device_eval_batch_size = 1,
    gradient_accumulation_steps=16,
    log_level="debug",
    save_strategy="epochs",
    logging_steps=25,
    learning_rate=1e-5,
    fp16= not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    eval_steps = 25,
    num_train_epochs =1,
    warmup_ratio=0.1,
    lr_scheduler_type="linear",
    dataset_text_field="text",
    max_seq_length=2048,
)

In [ ]:
trainer = SFTTrainer(
    model=model,train_dataset = ds['train'],eval_dataset=ds['test'],
    tokenizer = tokenizer,
    packing = True,
    args = training_arguments,
)

In [ ]:
gpu_stats = torch.cuda.get_device_properties(0)

start_gpu_memory = round(torch.cuda.max_memory_reserved()/1024/1024/1024,3)

max_memory = round(gpu_stats.total_memory / 1024 /1024 /1024,3)

trainer_ = trainer.train()

In [ ]:
!pip install unsloth

